# RNN Primer
### Advantages of sequential NNs over traditional ML model on time-series data

This notebook showcases the advantage of using RNN models (LSTMs, GRUs) over Decision Trees for a classification task on time series data.

### Task description
**General**: classify the transport mode of a device (user) given some sensor data.

**Specific**: we will do a **binary classification** between "walk" and "train" modes, which correspond to a person walking or being in a train.

### Input data
Let's assume we have locations of a device, and we're going to compute two features based on locations:
1. Speed at each timestep
2. Distance to a public transport stop at each timestep.

Each train or walk sample would consist of many timesteps, thus representing a time series sequence.

For demonstration purposes, the **features are generated syntetically** based on obervations from real sensor data.

In [1]:
import numpy as np
import plotly.express as px
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from rnnprimer.datagen import generate_train_sample, generate_walk_sample, Dataset

## Walk sample

Walk sample has distance and speed generated as follows:
* distance is a unform random number between 0 and 10000 (meters)
* speed is normally distributed with a mean at 5 and standard deviation at 2.5 (km/h)

In [17]:
sample = generate_walk_sample(seq_size=100)
fig.update_layout(title_text="Walk sample")
fig = sample.get_figure()
fig.show()

## Train sample with no outliers
Train sample has distance and speed generated as follows:
* distance linearly increases from 0 to 10000 (meters) and back to 0 along a segment
* speed quickly reaches 100 (km/h), stays contants, and then decreases to 0 again

Train sample can constist of one of more segments (5 on the image below)

In [10]:
sample = generate_train_sample(seg_size=100)
fig = sample.get_figure()
fig.update_layout(title_text="Train sample without outliers")
fig.show()

## Train sample with outliers

Outliers are introduced in the speed feature at a random point in time, with a given probability. Outlier has a speed drawn from the same distribution as that of a walk sample. 

In [12]:
sample = generate_train_sample(seg_size=100, outlier_prob=0.01)
fig = sample.get_figure()
fig.update_layout(title_text="Train sample with outliers")
fig.show()

## Traditional ML models
As an instance of a traditional ML model we're going to use a random forest classifier.

For training and evaluation, we are going to flatten our sequential samples and give features at each timestamp as individual training samples.

We then start to inroduce outliers with increasing probability in our train samples and measure what happens with **Precision** and **Recall** metrics.

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
data = []
from sklearn.model_selection import StratifiedShuffleSplit, KFold, cross_val_score
for outlier_prob in (0.01, 0.05, 0.10, 0.20, 0.30, 0.40, 0.50):
    X, y = Dataset.generate(train_outlier_prob=outlier_prob).get_flat_X_y()
    cv = StratifiedShuffleSplit(n_splits=2, test_size=0.2)
    clf = RandomForestClassifier(n_estimators=50, class_weight="balanced")
    precision = np.mean(cross_val_score(clf, np.array(X), y, cv=cv, scoring="precision", n_jobs=-1))
    recall = np.mean(cross_val_score(clf, np.array(X), y, cv=cv, scoring="recall", n_jobs=-1))
    data.append({'outlier_prob': outlier_prob, 'precision': precision, 'recall': recall})

In [30]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[d['outlier_prob'] for d in data], y=[d['precision'] for d in data],
                    name='precision'))
fig.add_trace(go.Scatter(x=[d['outlier_prob'] for d in data], y=[d['recall'] for d in data],
                    name='recall'))

fig.show()

## Test flat feature with a simple NN model

Currently doesn't work, why? It seems it settles on always predicting 1 for all samples.


1. works better when I increase the number of neurons in the input layer. Why? I thought input layer should have the same # of neurons as the # of features?
2. SGD optimizer doesn't work even when I increate the number of input neurons.

In [10]:
import tensorflow as tf
model = tf.keras.Sequential(
    [
            tf.keras.layers.Dense(2, activation="relu"),
            tf.keras.layers.Dense(16, activation="relu"),
            tf.keras.layers.Dense(1, activation="sigmoid")
    ]
)
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]
)

In [12]:
X, y = Dataset.generate(train_outlier_prob=0, n_samples=100).get_flat_X_y()

model.fit(
    x=X,
    y=y,
    batch_size=100,
    epochs=50,
    validation_split=0.1
)

Epoch 1/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6895 - recall_4: 0.0107 - precision_4: 1.0000 - val_loss: 0.6884 - val_recall_4: 0.0138 - val_precision_4: 1.0000
Epoch 2/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6893 - recall_4: 0.0435 - precision_4: 0.5590 - val_loss: 0.6881 - val_recall_4: 0.0146 - val_precision_4: 1.0000
Epoch 3/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6892 - recall_4: 0.0116 - precision_4: 1.0000 - val_loss: 0.6881 - val_recall_4: 0.0150 - val_precision_4: 1.0000
Epoch 4/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6891 - recall_4: 0.1743 - precision_4: 0.5149 - val_loss: 0.6876 - val_recall_4: 0.0158 - val_precision_4: 1.0000
Epoch 5/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6886 - recall_4: 0.0133 - precision_4: 0.9771 - val_loss: 0.6866 - val_recall_4: 0.0191 - val_precision_4: 1.0000
Epoch 6/50
450/450 [==============================

Epoch 44/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6877 - recall_4: 0.2612 - precision_4: 0.5085 - val_loss: 0.6846 - val_recall_4: 0.0244 - val_precision_4: 1.0000
Epoch 45/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6914 - recall_4: 0.1279 - precision_4: 0.5102 - val_loss: 0.6884 - val_recall_4: 0.0134 - val_precision_4: 1.0000
Epoch 46/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6896 - recall_4: 0.2164 - precision_4: 0.5086 - val_loss: 0.6885 - val_recall_4: 0.0134 - val_precision_4: 1.0000
Epoch 47/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6896 - recall_4: 0.1230 - precision_4: 0.5125 - val_loss: 0.6886 - val_recall_4: 0.0134 - val_precision_4: 1.0000
Epoch 48/50
450/450 [==============================] - 1s 1ms/step - loss: 0.6896 - recall_4: 0.1170 - precision_4: 0.5173 - val_loss: 0.6886 - val_recall_4: 1.0000 - val_precision_4: 0.4922
Epoch 49/50
450/450 [========================

## RNN model showcase: LSTM

Doesn't work at all, not sure why, gives an error about layer rank...\

In [13]:
import tensorflow as tf
model = tf.keras.Sequential(
    [
        tf.keras.layers.GRU(8, return_sequences=True, recurrent_dropout=0.5, input_shape=(None, 2)),
        tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(4, activation="relu", input_shape=(8,))
        ),
        tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(
                1, activation="sigmoid", input_shape=(4,), bias_initializer=None,
            )
        ),
    ]
)
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()],
    sample_weight_mode="temporal",
)

In [14]:
dataset = Dataset.generate(train_outlier_prob=0, n_samples=10)

model.fit(
    x=dataset.to_tfds(),
    epochs=50
)

Epoch 1/50


ValueError: in user code:

    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /usr/local/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:886 __call__
        self.name)
    /usr/local/lib/python3.7/site-pac